In [4]:
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
sqlite3.sqlite_version

In [ ]:
print('''
Есть таблица с данными по продажам кофе:
SALE_DTTM – точное время продажи,
CARD_NUMBER – номер скидочной карты
COFFEE_TYPE – название проданного кофейного напитка
PRICE – полная стоимость
DISCOUNT – размер скидки в %
GROCERY_FLG – флаг покупки выпечки в том же чеке

Составьте алгоритм SQL-запросов для решения следующих заданий:

1) Вычислить медианное значение суммарных месячных трат постоянных клиентов за каждый месяц
(при отсутствии встроенной функции, вычисляющей медиану).

2) Вычислить максимальное количество последовательных дней, когда один и тот же клиент покупал кофе с выпечкой.

3) В кофейне есть акция – при ежедневной покупке кофе с выпечкой можно купить термокружку со скидкой.
При этом размер скидки на термокружку получается путём последовательного применения всех скидок на кофе
с выпечкой, применённых за накопленную на момент покупки термокружки серию ежедневных покупок кофе с
выпечкой.Вычислить максимальную скидку на термокружку, которую мог достичь клиент (размер скидки клиента
не является константой).
''')

In [11]:
df = pd.DataFrame({'dt': pd.date_range('2020-04-01', '2020-06-31')})
# df['num'] = np.random.poisson(30, len(df))
df

ValueError: could not convert string to Timestamp

In [ ]:
df = pd.DataFrame({'sale_dttm':(df['dt'].apply(lambda x: [x])*df['num']).sum()})

In [ ]:
df['card_number'] = pd.Series(list(range(40)) + [np.nan]*10).sample(len(df), replace=True).values

In [ ]:
df['coffee_type'] = pd.Series(['espesso', 'americano', 'capuccino','capuccino','capuccino','latte']).sample(len(df), replace=True).values

In [ ]:
coffee_price =  pd.DataFrame({'espesso': [100], 'americano': [120], 'capuccino': [130],'latte': [150]}).T.reset_index()

In [12]:
coffee_price.columns = ['coffee_type', 'coffee_price']
coffee_price

NameError: name 'coffee_price' is not defined

In [ ]:
df = df.merge(coffee_price,how='left', on='coffee_type')

In [ ]:
df['discount'] = pd.Series([0, 0, 0, 0, 0, 0.05, 0.05,0.05,0.05,0.1,0.15]).sample(len(df), replace=True).values

In [ ]:
df['grocery_flg'] = pd.Series([0,0,0,1]).sample(len(df), replace=True).values

In [ ]:
df['price'] = (df['coffee_price'] + df['grocery_flg']*100)*(1-df['discount'])

In [ ]:
df.to_csv("path_or_buf", header=True, index=True)

In [1]:
from sqlalchemy import create_engine, MetaData

In [2]:
engine = create_engine('sqlite:///data/db1', convert_unicode=True)
metadata = MetaData(bind=engine)

In [ ]:
engine.execute("select * from sales_day_1")

In [8]:
sql = '''select t.card_number from sales_day_1 t'''
select(sql)

,dt,card_number,asgrocery_flg,num_purchases,discount
0,2020-04-01,NaN,1,5,0.05
1,2020-04-01,2.0,0,1,0.00
2,2020-04-01,3.0,1,1,0.00
3,2020-04-01,6.0,0,1,0.00
4,2020-04-01,9.0,1,1,0.00
...,...,...,...,...,...
1132,2020-05-31,31.0,1,1,0.05
1133,2020-05-31,32.0,0,1,0.00
1134,2020-05-31,33.0,0,1,0.00
1135,2020-05-31,36.0,0,1,0.00


In [5]:
con = sqlite3.connect('data/db1')

In [6]:
cur = con.cursor()

In [ ]:
df.to_sql('coffee_sales', con, index=False, if_exists='replace')

In [ ]:
sql = '''select * from coffee_sales t limit 10'''

In [7]:
def select(sql):
    return pd.read_sql(sql, con)

In [ ]:
select(sql)

In [ ]:
sql = '''
select date(t.sale_dttm, 'start of month') as month , t.card_number, sum(t.price) as price_sum
from coffee_sales t
group by  date(t.sale_dttm, 'start of month'), t.card_number
group by t.
'''

select(sql)

In [ ]:
df['sale_dttm'].values.astype('datetime64[M]')

In [ ]:
df['month'] = df['sale_dttm'].values.astype('datetime64[M]')

In [ ]:
df[:3]

In [ ]:
m = df.groupby(['month', 'card_number'])['price'].sum().reset_index()
m

In [ ]:
m.groupby('month')['price'].agg(['count', 'median'])

In [ ]:
sql = '''
select x.month,
       avg(x.price_sum) as Median

from (select t.*,
             row_number() over (partition by t.month order by t.price_sum) as RowNum,
             count(*) over (partition by t.month)                          as RowCnt

      from (select date(t.sale_dttm, 'start of month') as month, t.card_number, sum(t.price) as price_sum
            from coffee_sales t
            group by date(t.sale_dttm, 'start of month'), t.card_number) t) x

where x.RowNum in ((x.RowCnt + 1) / 2, (x.RowCnt + 2) / 2)

group by x.month
'''

select(sql)

In [ ]:
sql ='''
drop table IF EXISTS sales_day_1;
create table sales_day_1 as

select date(t.sale_dttm)                                           as dt,
       t.card_number,
       max(t.grocery_flg)                                             asgrocery_flg,
       count(1)                                                    as num_purchases,
       max(case when t.grocery_flg = 1 then t.discount else 0 end) as discount
from coffee_sales t
group by date(t.sale_dttm), t.card_number
;
'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql ='''
select

from coffee_sales t

'''

select(sql)

In [ ]:
sql ='''
select *
       
from coffee_sales t

'''

select(sql)

2) Вычислить максимальное количество последовательных дней, когда один и тот же клиент покупал кофе с выпечкой.

In [ ]:
sql ='''select
  date(t.sale_dttm) as dt,
  t.card_number,
  count(1) as num_purchace,
  max(t.grocery_flg) as grocery_flg
    from coffee_sales t
where t.card_number = 39
group by t.sale_dttm, t.card_number
'''
df = select(sql)
df.head()

In [ ]:
sql ='''select * from coffee_sales t
where t.card_number=39 and date(t.sale_dttm)='2020-04-01' '''
select(sql)

In [ ]:
df.to_csv('path_or_buf', header=True, index=True)

In [2]:
import numpy as np

np.linspace(2, 3, num=30, endpoint=False, axis=0)

array([2.        , 2.03333333, 2.06666667, 2.1       , 2.13333333,
       2.16666667, 2.2       , 2.23333333, 2.26666667, 2.3       ,
       2.33333333, 2.36666667, 2.4       , 2.43333333, 2.46666667,
       2.5       , 2.53333333, 2.56666667, 2.6       , 2.63333333,
       2.66666667, 2.7       , 2.73333333, 2.76666667, 2.8       ,
       2.83333333, 2.86666667, 2.9       , 2.93333333, 2.96666667])

In [4]:
import pandas as pd

df = pd.DataFrame({'d1': [1,2,3], 'd2': [4,5,6]})
df.drop('d1', axis=1, inplace=True)

In [5]:
df

,d2
0,4
1,5
2,6


In [ ]:
sql ='''select * from coff t'''
select(sql)